In [316]:
import pandas as pd
import numpy as np

candles = pd.read_csv("XBTUSD-1m-data.csv")
candle = pd.Series()
#print(candles)

# iterate over rows with iterrows()
for index, data in candles.head(500).iterrows():
     #determine if candles are of opposite type
     if data['open'] < data['close']:
        type = "green"
     elif data['open'] > data['close']:
        type = "red"
     else:
        type = "abs_doji"
     #append size
     size = [abs(data['open']-data['close']), type]
     candle.at[index] = size

#print(candle.iloc[-1][-1])
#print(candle.iloc[-2][-1])

#realtime func
def tradesignal(i = -1):
    if candle.iloc[i][-1] == candle.iloc[i-1][-1]:
        print("WAIT")
    else:
        if candle.iloc[i][-1] == "red":
            print("SELL")
        elif candle.iloc[i][-1] == "green":
            print("BUY")
        else:
            print("WAIT")

cseries = iter(candle.index[::-1])
next(cseries)

for i in cseries:
    i = -1*(i)
    #print(i)
    tradesignal(i)

C:\Users\eyese\.julia\conda\3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  """


WAIT
SELL
BUY
WAIT
SELL
WAIT
BUY
WAIT
WAIT
SELL
WAIT
WAIT
WAIT
WAIT
BUY
SELL
WAIT
WAIT
BUY
WAIT
WAIT
SELL
BUY
SELL
BUY
WAIT
WAIT
SELL
WAIT
WAIT
BUY
WAIT
SELL
BUY
WAIT
WAIT
BUY
SELL
BUY
WAIT
BUY
WAIT
WAIT
WAIT
SELL
BUY
SELL
WAIT
BUY
WAIT
BUY
WAIT
SELL
WAIT
WAIT
SELL
WAIT
WAIT
BUY
WAIT
WAIT
WAIT
WAIT
SELL
BUY
WAIT
SELL
WAIT
WAIT
WAIT
WAIT
WAIT
BUY
WAIT
SELL
BUY
SELL
WAIT
SELL
BUY
WAIT
WAIT
SELL
WAIT
BUY
WAIT
WAIT
WAIT
SELL
WAIT
WAIT
WAIT
BUY
WAIT
WAIT
WAIT
WAIT
WAIT
WAIT
WAIT
WAIT
SELL
WAIT
WAIT
SELL
BUY
SELL
BUY
WAIT
WAIT
WAIT
BUY
WAIT
WAIT
SELL
BUY
SELL
BUY
WAIT
WAIT
BUY
WAIT
WAIT
WAIT
WAIT
WAIT
BUY
SELL
WAIT
WAIT
WAIT
WAIT
WAIT
BUY
SELL
BUY
WAIT
WAIT
WAIT
WAIT
WAIT
WAIT
WAIT
BUY
WAIT
WAIT
BUY
SELL
BUY
WAIT
WAIT
WAIT
SELL
BUY
WAIT
WAIT
SELL
BUY
SELL
BUY
SELL
WAIT
BUY
SELL
BUY
SELL
BUY
WAIT
WAIT
WAIT
SELL
BUY
SELL
BUY
WAIT
WAIT
BUY
SELL
WAIT
BUY
SELL
WAIT
WAIT
BUY
WAIT
WAIT
SELL
WAIT
WAIT
WAIT
BUY
WAIT
BUY
WAIT
BUY
SELL
BUY
SELL
WAIT
WAIT
BUY
WAIT
WAIT
SELL
BUY
SELL
WAIT
BUY
WAIT
SELL
W